### How to run:

1. Download model_parse_spring-v0_1_0 and model_generate_t5-v0_1_0 from https://amrlib.readthedocs.io/en/latest/install/
2. Extract 
    - generator model files into ./anaconda3/python3.8/site-packages/amrlib/data/model_gtos/ (default for amrlib)
    - parser model files into ./anaconda3/python3.8/site-packages/amrlib/data/model_stog/ (default for amrlib)


# 1. Text-to-AMR Parsing

In [2]:
import amrlib

#p = '/Users/chris/Code/model_parse_spring-v0_1_0'
#g = '/Users/chris/Code/model_generate_t5-v0_1_0'

In [7]:
### Parsing

stog = amrlib.load_stog_model(model_dir=None)
#graphs = stog.parse_sents(['some 200,000 visitors were expected at the second cebit home , 10,000 less than two years ago .'])
graphs = stog.parse_sents(['the man sing a song'], add_metadata = True) #metadata can be switched on and off

for graph in graphs:
    print(graph)

# ::snt the man sing a song
(z0 / sing-01
      :ARG0 (z1 / man)
      :ARG1 (z2 / song))


In [8]:
graphs2 = stog.parse_sents(['The president spoke in March. He wanted to address the state.'], add_metadata = False)

In [10]:

for graph in graphs:
    print(graph)

print('-------------------------------')

    
for graph in graphs2:
    print(graph)

# ::snt the man sing a song
(z0 / sing-01
      :ARG0 (z1 / man)
      :ARG1 (z2 / song))
-------------------------------
(z0 / multi-sentence
      :snt1 (z1 / speak-01
            :ARG0 (z2 / person
                  :ARG0-of (z3 / have-org-role-91
                        :ARG2 (z4 / president)))
            :time (z5 / date-entity
                  :month 3))
      :snt2 (z6 / want-01
            :ARG0 (z7 / he)
            :ARG1 (z8 / address-03
                  :ARG0 z7
                  :ARG1 (z9 / state))))


In [11]:
print(type(graph))
print((graph))

<class 'str'>
(z0 / multi-sentence
      :snt1 (z1 / speak-01
            :ARG0 (z2 / person
                  :ARG0-of (z3 / have-org-role-91
                        :ARG2 (z4 / president)))
            :time (z5 / date-entity
                  :month 3))
      :snt2 (z6 / want-01
            :ARG0 (z7 / he)
            :ARG1 (z8 / address-03
                  :ARG0 z7
                  :ARG1 (z9 / state))))


# 2. Generating

In [4]:
#Load generator model 
gtos = amrlib.load_gtos_model() #load from default folder; see how to run



In [43]:
### Input for Generator seems to be graphs without meta data; either in string or penman object format


In [12]:
sents, _ = gtos.generate(graphs2, disable_progress=False)
for sent in sents:
    print(sent)

for gnum in range(len(graphs2)):
    print('graph number', gnum)
    for sent in gtos.get_ans_group(sents, gnum):
        print(sent)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.66s/it]

The President spoke in March. He wanted to address the state.
graph number 0
The President spoke in March. He wanted to address the state.


# Useful amrlib functionalities

In [51]:
#Automatically parse multiple sentences into separate AMRs using spacy

import amrlib
import spacy
amrlib.setup_spacy_extension()
nlp = spacy.load('en_core_web_sm')
doc = nlp('This is a test of the SpaCy extension. The test has multiple sentences.')
graphs = doc._.to_amr()
for graph in graphs:
    print(graph)

# ::snt This is a test of the SpaCy extension.
(z0 / test-01
      :ARG0 (z1 / this)
      :ARG1 (z2 / extend-01
            :ARG1 (z3 / program
                  :name (z4 / name
                        :op1 "SpaCy"))))
# ::snt The test has multiple sentences.
(z0 / have-03
      :ARG0 (z1 / test)
      :ARG1 (z2 / sentence
            :quant (z3 / multiple)))


In [70]:
### Visualizing

### Only runs withouth metadata in heading
### Step 1: install graphviz package
### Step 2: 

from   amrlib.graph_processing.amr_plot import AMRPlot
from   amrlib.graph_processing.amr_loading import load_amr_entries

input_file = '/Users/chris/opt/anaconda3/lib/python3.8/site-packages/amrlib/data/amr-bank-struct-v3.0.txt'
# Load the AMR file

entries = load_amr_entries(input_file)
entry = entries[12]    # pick an index
        
# Plot
plot = AMRPlot()
plot.build_from_graph(graphs2, debug=True)
plot.view()

Instance(source='z0', role=':instance', target='multi-sentence')
Instance(source='z1', role=':instance', target='speak-01')
Instance(source='z2', role=':instance', target='person')
Instance(source='z3', role=':instance', target='have-org-role-91')
Instance(source='z4', role=':instance', target='president')
Instance(source='z5', role=':instance', target='date-entity')
Instance(source='z6', role=':instance', target='want-01')
Instance(source='z7', role=':instance', target='he')
Instance(source='z8', role=':instance', target='address-03')
Instance(source='z9', role=':instance', target='state')
Edge(source='z0', role=':snt1', target='z1')
Edge(source='z1', role=':ARG0', target='z2')
Edge(source='z2', role=':ARG0-of', target='z3')
Edge(source='z3', role=':ARG2', target='z4')
Edge(source='z1', role=':time', target='z5')
Edge(source='z0', role=':snt2', target='z6')
Edge(source='z6', role=':ARG0', target='z7')
Edge(source='z6', role=':ARG1', target='z8')
Edge(source='z8', role=':ARG0', target=

In [1]:
### Additional notes
### Base-baseline idea:

### 1. Take a multiple sentences. Select random sentences. Generate multi-sentence AMR
### -> basically everything available

### 2. Merge AMRs and then generate them
### -> use penman format so we can manipulate them? find concepts that exist in every AMR
### -> use token (from each zX-nodes) to create a new multi-sentence summary graph (second parse)
### -> generate source-amr to text

### Optional: try out coref github for amrlib: https://github.com/bjascob/amr_coref/tree/master/amr_coref/coref